引入相关包

In [1]:
import sys
import os
import numpy as np
from datetime import datetime
import glob
#Get the path of the current file
current_path = os.getcwd()
# Add the evoman_framework folder to sys.path
sys.path.append(os.path.join(current_path, "evoman"))
# Now you can import contents from the evoman environment
from evoman.environment import Environment
from evoman.controller import Controller
# Initialize environment


pygame 2.6.0 (SDL 2.28.4, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


创建目录

In [2]:

# Set headless mode to improve running speed
headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

experiment_name = "single_island_optimization"
test_log_path = 'single_island_optimization/test_logs/'
module_path = 'single_island_optimization/models/' 
if not os.path.exists(experiment_name):
    os.makedirs(experiment_name)

if not os.path.exists(test_log_path):
    os.makedirs(test_log_path)

if not os.path.exists(module_path):
    os.makedirs(module_path)


定义控制器

In [3]:
# Define Sigmoid activation function
def sigmoid_activation(x):
    return 1.0 / (1.0 + np.exp(-x))
# Custom controller with a simple neural network structure
class player_controller(Controller):
    def __init__(self, _n_hidden):
        self.n_hidden = [_n_hidden]  # Number of neurons in the hidden layer

    def set(self, controller, n_inputs):
        if self.n_hidden[0] > 0:
            # Extract weights and biases from controller parameters to build the neural network
            self.bias1 = controller[: self.n_hidden[0]].reshape(1, self.n_hidden[0])
            weights1_slice = n_inputs * self.n_hidden[0] + self.n_hidden[0]
            self.weights1 = controller[self.n_hidden[0] : weights1_slice].reshape(
                (n_inputs, self.n_hidden[0])
            )
            self.bias2 = controller[weights1_slice : weights1_slice + 5].reshape(1, 5)
            self.weights2 = controller[weights1_slice + 5 :].reshape(
                (self.n_hidden[0], 5)
            )

    def control(self, inputs, controller):
        # Normalize inputs
        inputs = (inputs - min(inputs)) / float((max(inputs) - min(inputs)))
        if self.n_hidden[0] > 0:
            output1 = sigmoid_activation(inputs.dot(self.weights1) + self.bias1)
            output = sigmoid_activation(output1.dot(self.weights2) + self.bias2)[0]
        else:
            bias = controller[:5].reshape(1, 5)
            weights = controller[5:].reshape((len(inputs), 5))
            output = sigmoid_activation(inputs.dot(weights) + bias)[0]

        # Determine player actions based on output
        left = 1 if output[0] > 0.5 else 0
        right = 1 if output[1] > 0.5 else 0
        jump = 1 if output[2] > 0.5 else 0
        shoot = 1 if output[3] > 0.5 else 0
        release = 1 if output[4] > 0.5 else 0

        return [left, right, jump, shoot, release]


# Initialize controller
player_controller_instance = player_controller(10)  # Example with 10 hidden neurons


定义相关函数

In [4]:

# Run simulation and return fitness
def simulation(env, x):
    #env.player_controller.set(x, n_inputs)  # Set controller parameters
    f, _, _, _ = env.play(pcont=x)
    return f

# Evaluate population
def evaluate(x):
    return np.array([simulation(env, ind) for ind in x])

# Initialize population
def initialize_population():
    pop = np.random.uniform(dom_l, dom_u, (npop, n_vars))
    fit_pop = evaluate(pop)
    return pop, fit_pop

# Uniform crossover
def uniform_crossover(p1, p2):
    offspring = np.zeros_like(p1)
    for i in range(len(p1)):
        if np.random.rand() < 0.5:
            offspring[i] = p1[i]
        else:
            offspring[i] = p2[i]
    return offspring

# Fixed mutation
def mutation_operation(offspring):
    for i in range(len(offspring)):
        if np.random.uniform(0, 1) <= mutation:
            offspring[i] += np.random.normal(0, 1)
    return offspring

# Crossover and mutation operations
def crossover_and_mutation(pop):
    total_offspring = np.zeros((0, n_vars))
    for p in range(0, pop.shape[0], 2):
        p1 = pop[np.random.randint(0, pop.shape[0])]
        p2 = pop[np.random.randint(0, pop.shape[0])]

        # Uniform crossover
        offspring = uniform_crossover(p1, p2)

        # Fixed mutation operation
        offspring = mutation_operation(offspring)

        # Fitness constraint
        offspring = np.clip(offspring, dom_l, dom_u)
        total_offspring = np.vstack((total_offspring, offspring))

    return total_offspring

# Elitism selection
def elitism(pop, fit_pop, elite_size):
    elite_indices = np.argsort(fit_pop)[-elite_size:]
    elite_pop = pop[elite_indices]
    elite_fitness = fit_pop[elite_indices]
    return elite_pop, elite_fitness


# Genetic algorithm evolution process
def evolve_population(pop, fit_pop):
    # Elitism selection
    elite_size = int(elitism_rate * npop)
    elite_pop, elite_fitness = elitism(pop, fit_pop, elite_size)

    # Crossover and mutation
    offspring = crossover_and_mutation(pop)
    fit_offspring = evaluate(offspring)

    # Combine elites and offspring
    pop = np.vstack((elite_pop, offspring[: npop - elite_size]))
    fit_pop = np.hstack((elite_fitness, fit_offspring[: npop - elite_size]))

    return pop, fit_pop

# Save final solution
def save_final_solution(
    pop, fit_pop, experiment_name, enemies, npop, gens, mutation, elitism_rate
):
    best_index = np.argmax(fit_pop)
    best_solution = pop[best_index]
    best_fitness = fit_pop[best_index]

    # Get current timestamp
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Filename uses the best fitness, population size, generations, enemy type, mutation rate, and elitism rate
    filename = (
        f"{experiment_name}/models/SIModel_fitness{best_fitness:.6f}_npop{npop}_gens{gens}_"
        f"enemy{enemies}_mut{mutation:.2f}_elitism{elitism_rate:.2f}_{current_time}.txt"
    )

    # Save the best individual's controller weights and biases
    np.savetxt(filename, best_solution)

    # Print information about the saved file
    print(f"Best solution saved as {filename} with fitness: {best_fitness:.6f}")

# Run evolution process
def run_evolution():
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{experiment_name}/test_logs/SI_log_{current_time}.txt"

    with open(filename, "a") as file:
        # Write header row
        file.write(
            f"npop{npop}_gens{gens}_"
            f"enemy{env.enemies}_mut{mutation:.2f}_elitism{elitism_rate:.2f}_{current_time}\n"
        )
        file.write("gen best mean std\n")

        pop, fit_pop = initialize_population()

        for generation in range(gens):
            pop, fit_pop = evolve_population(pop, fit_pop)

            # Record results
            best = np.max(fit_pop)  # Best value
            mean = np.mean(fit_pop)  # Mean value
            std = np.std(fit_pop)  # Standard deviation

            result = f"{generation} {best:.6f} {mean:.6f} {std:.6f}"
            print(result)

            # Write to file
            file.write(result + "\n")

        # Save the final best solution
        save_final_solution(
            pop, fit_pop, experiment_name, env.enemies, npop, gens, mutation, elitism_rate
        )


# Load best solution
def load_best_solution(filepath):
    return np.loadtxt(filepath)

# Simulation test
def simulation_test(env, x):
    env.player_controller.set(x, n_inputs)  # Set controller parameters
    f, player_life, enemy_life, _ = env.play(pcont=x)  # Return player and enemy life
    return f, player_life, enemy_life


初始化环境

In [5]:

env = Environment(
    experiment_name=experiment_name,
    multiplemode="yes",
    playermode="ai",
    player_controller=player_controller_instance,
    enemymode="static",
    level=2,
    speed="fastest",
    visuals=False,
)


MESSAGE: Pygame initialized for simulation.


超参数

In [6]:
#set your enemy here
env.enemies=[1,2,3,4,5,6,7,8]

# Genetic algorithm parameters
n_inputs = 20  # Number of inputs to the controller
n_vars = (
    n_inputs * 10 + 10 + 10 * 5 + 5
)  # Total number of weights and biases in the controller
dom_u = 1
dom_l = -1
npop = 120  # Population size
gens = 500  # Total generations
mutation = 0.3  # Mutation rate
elitism_rate = 0.15  # Elitism rate


训练

In [13]:
run_evolution()

0 13.801356 -4.062424 5.454370
1 13.801356 -1.690942 7.116880
2 13.801356 0.086321 7.625039
3 14.295904 1.531739 8.053026
4 14.295904 2.581885 8.047549
5 17.094023 2.936592 8.844591
6 17.094023 3.208290 8.549139
7 24.242326 5.408661 9.145196
8 24.242326 4.203974 10.395165
9 24.977501 6.976892 10.061679
10 24.977501 7.367602 10.014916
11 24.977501 6.975443 10.493207
12 24.977501 7.339712 10.779896
13 24.977501 7.537445 10.833656
14 29.174715 7.125467 12.126259
15 29.174715 7.975314 11.782594
16 29.174715 8.255083 11.840301
17 29.174715 10.055782 11.041016
18 29.174715 11.363090 10.194255
19 29.174715 10.340274 12.235599
20 29.174715 9.023170 12.530093
21 29.174715 8.658376 12.894639
22 29.174715 9.708527 12.686331
23 29.174715 10.214811 11.334288
24 29.174715 9.421900 12.518808
25 29.174715 12.673293 10.983690
26 29.174715 11.211506 12.138440
27 29.174715 8.925627 12.896941
28 29.174715 10.563126 13.305689
29 29.174715 10.488167 13.255073
30 29.174715 11.292143 12.163460
31 30.174568 11

测试

In [12]:
def test_loaded_model():
    # Hardcoded model file path
    filepath = f"{experiment_name}/models/DEModel_fitness54.347888_npop100_gens500_enemy[1, 2, 3, 4, 5, 6, 7, 8]_mut0.20_elitism0.20_20241018_133158.txt"

    fitness_scores = []
    gains = []  # Used to store individual_gain for each run

    # Load the saved model
    solution = load_best_solution(filepath)

    # Run 5 times
    for i in range(5):
        fitness, player_life, enemy_life = simulation_test(env, solution)
        fitness_scores.append(fitness)

        # Calculate individual_gain
        individual_gain = player_life - enemy_life
        gains.append(individual_gain)

        print(f"Run {i+1} fitness: {fitness}, individual_gain: {individual_gain}")

    # Calculate the average fitness
    average_fitness = np.mean(fitness_scores)
    average_gain = np.mean(gains)

    print(f"\nAverage fitness over 5 runs: {average_fitness}")
    print(f"Average individual gain over 5 runs: {average_gain}")


# # Run test
# test_loaded_model()




In [27]:
import glob
import numpy as np
import csv
import os

def evaluate_models_to_csv(experiment_name, enemy):
    # Get all model files
    model_files = glob.glob(f"{experiment_name}/models/E{enemy}1020/*.txt")
    print(f"Found model files: {model_files}")  # Debugging line

    # Check if there are any model files
    if not model_files:
        raise FileNotFoundError(
            f"No model files found in {experiment_name}/models/E{enemy}/"
        )

    output_csv = f"{experiment_name}_enemy{enemy}.csv"  # Output CSV file

    results = []

    for filepath in model_files:
        # Load the saved model
        solution = load_best_solution(filepath)
        print(f"Loaded solution from {filepath}")  # Debugging line

        # Run 1 time
        fitness, player_life, enemy_life = simulation_test(env, solution)
        print(f"Test results - Fitness: {fitness}, Player Life: {player_life}, Enemy Life: {enemy_life}")  # Debugging line

        # Calculate individual_gain
        individual_gain = player_life - enemy_life

        print(
            f"Model: {filepath}, fitness: {fitness}, individual_gain: {individual_gain}"
        )

        # Store the result
        results.append([filepath, fitness, individual_gain])

    # Write results to CSV
    with open(output_csv, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Model", "Fitness", "Individual Gain"])
        writer.writerows(results)

    print(f"Results have been written to {output_csv}")


In [30]:
env.enemies = [1, 2, 3, 4, 5, 6, 7, 8]
# test_loaded_model()
evaluate_models_to_csv(experiment_name, 12345678)

Found model files: ['single_island_optimization/models/E123456781020/SIModel_fitness45.070156_npop120_gens500_enemy[1, 2, 3, 4, 5, 6, 7, 8]_mut0.20_elitism0.20_20241020_115351.txt', 'single_island_optimization/models/E123456781020/SIModel_fitness42.161660_npop120_gens500_enemy[1, 2, 3, 4, 5, 6, 7, 8]_mut0.20_elitism0.20_20241020_132944.txt', 'single_island_optimization/models/E123456781020/SIModel_fitness43.562386_npop120_gens500_enemy[1, 2, 3, 4, 5, 6, 7, 8]_mut0.20_elitism0.20_20241020_151037.txt', 'single_island_optimization/models/E123456781020/SIModel_fitness45.715758_npop120_gens500_enemy[1, 2, 3, 4, 5, 6, 7, 8]_mut0.20_elitism0.20_20241020_121755.txt', 'single_island_optimization/models/E123456781020/SIModel_fitness40.426768_npop120_gens500_enemy[1, 2, 3, 4, 5, 6, 7, 8]_mut0.20_elitism0.20_20241020_153624.txt', 'single_island_optimization/models/E123456781020/SIModel_fitness40.743907_npop120_gens500_enemy[1, 2, 3, 4, 5, 6, 7, 8]_mut0.20_elitism0.20_20241020_144500.txt', 'single_